In [11]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
!pip install setfit

In [13]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# CA POC

In [14]:
from crewai import Agent, Task, Crew

In [15]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

In [17]:
import torch
import pysbd

from setfit import SetFitModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from crewai_tools import BaseTool


class LexGlueUnfairClausesTool(BaseTool):
    name: str = "LexGLUE"
    description: str = "Identifies potentially unfair clauses in online terms of service"

    def __init__(self):
        super().__init__()
        self._tokenizer = AutoTokenizer.from_pretrained('marmolpen3/lexglue-unfair-tos')
        self._model = AutoModelForSequenceClassification.from_pretrained('marmolpen3/lexglue-unfair-tos')

    def _run(self, tems_of_service: str) -> str:
        # split input text into sentences
        seg = pysbd.Segmenter(language="en", clean=False)

        # detect potential unfair clauses with LEXGLUE
        sentences_list = seg.segment(tems_of_service)

        input_ids = self._tokenizer(sentences_list, padding=True, truncation=True, return_tensors="pt")
        outputs = self._model(**input_ids)
        results = torch.sigmoid(outputs[0]).tolist()

        # generate a markdown text with the results. For each sentence, print this:
        # Clause: "the cluase"
        # Unfainesss: comma separated list of unfairness categories
        markdown = ""
        for i, _ in enumerate(results):
            markdown += f"**Clause:** {sentences_list[i]}\n"

            # list of categories
            categories = [
                "Limitation of liability" if results[i][0] >= 0.5 else "",
                "Unilateral termination" if results[i][1] >= 0.5 else "",
                "Unilateral change" if results[i][2] >= 0.5 else "",
                "Content removal" if results[i][3] >= 0.5 else "",
                "Contract by using" if results[i][4] >= 0.5 else "",
                "Choice of law" if results[i][5] >= 0.5 else "",
                "Jurisdiction" if results[i][6] >= 0.5 else "",
                "Arbitration" if results[i][7] >= 0.5 else ""
            ]

            markdown += f"**Unfairness:** {', '.join([c for c in categories if c])}\n\n"
            return markdown

class ICanTool(BaseTool):
    name: str = "iCan"
    description: str = "Generates a list with all rights and obligations in online terms of service"

    def __init__(self):
        super().__init__()
        self._model = SetFitModel.from_pretrained('marmolpen3/paraphrase-mpnet-base-v2-sla-obligations-rights')

    def _run(self, tems_of_service: str) -> str:
        # split input text into sentences
        seg = pysbd.Segmenter(language="en", clean=False)

        sentences_list = [s for s in seg.segment(tems_of_service) if len(s.split(" ")) > 5]

        if len(sentences_list) == 0:
            return "No rights or obligations found in the text."

        predictions = self._model(sentences_list)
        results = {"obligations": [], "rights": [], "none": []}
        label_to_key = {0: "obligations", 1: "rights", 2: "none"}

        for i, _ in enumerate(predictions):
            key = label_to_key[int(predictions[i])]
            sentence = sentences_list[i]
            results[key].append(sentence)

        # generate a markdown text with the results. Print the rights and obligations
        markdown = ""
        for key in results:
            if key != "none":
                markdown += f"**{key.capitalize()}**\n"
                for sentence in results[key]:
                    markdown += f"- {sentence}\n"
                markdown += "\n"

        return markdown

class ReadGithubTosTool(BaseTool):
    name: str = "GithubTos"
    description: str = "Read Github's terms of service"

    def _run(self, argument: str) -> str:
        with open("../github-terms-of-service.md", "+r") as f:
            markdown = f.read()
        return markdown

In [ ]:
lexglue_tool = LexGlueUnfairClausesTool()
ican_tool = ICanTool()
github_tool = ReadGithubTosTool()

In [19]:
legal_expert = Agent(
    role="Legal Expert",
    goal="To provide accurate and comprehensive answers to legal questions",
    tools=[lexglue_tool, ican_tool],
    verbose=True,
    backstory=(
        "As a Legal Expert, your role is to provide accurate "
        "and comprehensive answers to legal questions. You are "
        "skilled in conducting legal research and analysis, "
        "interpreting statutes and case law, and providing "
        "clear and concise explanations of complex legal issues."
    )
)

In [26]:
# document RAG tool
from crewai_tools import MDXSearchTool

inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask about the agreement below:\n"
	    "Question: {inquiry}\n\n"
        "Agreement: {agreement}\n\n"
		"Make sure to use everything you know to provide the best support possible."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, if you use an external solution, "
        "it should include all the content you it returns, and maintain a helpful and friendly "
		    "tone throughout."
    ),
	tools=[lexglue_tool, ican_tool],
    agent=legal_expert,
)

In [27]:
crew = Crew(
  agents=[legal_expert],
  tasks=[inquiry_resolution],
  verbose=2,
  memory=True
)

In [35]:
inputs = {
    "customer": "AYESA",
    "inquiry": "Extract all unfair terms from the GitHub terms of service.",
    "agreement": """Version: December 2022 1 (6)
                            GitHub Customer Agreement
                            This Agreement consists of the General Terms, the applicable Product Specific Terms and any additional terms
                            GitHub presents when an Order is placed. The Agreement takes effect when Customer accepts the General
                            Terms, and the individual who accepts these General Terms represents that they are authorized to enter into
                            this Agreement on behalf of Customer.
                            GitHub General Terms
                            These General Terms apply to all of Customer’s Orders under this Agreement. Capitalized terms have the
                            meanings given under Definitions.
                            1 License to use GitHub Products
                            1.1 License grant. Products are licensed and not sold. Subject to Customer’s compliance with this
                            Agreement, GitHub grants to Customer a nonexclusive and limited license to install and use the
                            Products ordered as provided in the applicable Product Specific Terms and this Agreement. The licenses
                            are only for Customer’s internal business purposes and are non-transferable except as expressly
                            permitted under this Agreement or applicable law.
                            1.2 Duration of licenses. Licenses expire at the end of the applicable Subscription Term unless renewed.
                            1.3 Accounts. Customer may assign each Subscription License to one individual End User for use or access
                            on any number of devices. Customer may not reassign a Subscription License to another End User
                            within 90 days of the last assignment, except where End User’s relationship with Customer ends or End
                            User goes on leave. End User accounts may not be shared by individuals.
                            1.4 End Users. Customer controls access to and use of the Products by End Users. Customer is responsible
                            for End Users’ use.
                            1.5 Product Specific Terms. Product Specific Terms apply to Products such as, but not limited to, GitHub AE,
                            GitHub Enterprise Cloud and GitHub Enterprise Server. If there is a conflict between the Product
                            Specific Terms and the General Terms, the Product Specific Terms apply for that Product.
                            1.6 Previews. Previews are provided “AS-IS”, “WITH ALL FAULTS” and “AS AVAILABLE”. GitHub may change
                            or discontinue Previews at any time without notice.
                            1.7 Product changes. GitHub has the right to make changes to the Products if such changes do not
                            materially lessen the Product’s functionality. GitHub may provide additional terms that apply to
                            Customer’s use of updates, new features, or related software.
                            1.8 Affiliates. Customer’s Affiliates may use the Products under this Agreement. Customer isresponsible for
                            its Affiliates’ use, and Customer has the sole right to enforce this Agreement.
                            1.9 Compliance with laws. Customer’s use of the Products must not violate any applicable laws, including
                            copyright or trademark laws, export control laws and regulations, including laws and regulations in its
                            jurisdiction.
                            1.10 Reservation of rights. Products are protected by copyright and other intellectual property laws and
                            international treaties. GitHub reserves all rights not expressly granted in this Agreement, and no rights
                            are granted or implied by waiver or estoppel.
                            1.11 Feedback. Feedback by Customer is optional and voluntary. Feedback may be used by GitHub for
                            any purpose without obligation of any kind as long as the Customer’s confidential information
                            remains confidential.
                            1.12 Restrictions. Unless expressly permitted in this Agreement or by law, Customer may not:
                            (a) reverse engineer, decompile, or disassemble any Product, or try to do so;
                            (b) run, upgrade or downgrade, or transfer parts of a Product separately at different times or on
                            different devices;
                            (c) install, use, or distribute other software or technology in any way that makes
                            GitHub’sintellectual property or technology subject to any other license terms;
                            (d) work around technical limitationsin a Product or restrictionsin Product documentation; or
                            (e) sell, rent, lease, sublicense, distribute or lend any Products to others, in whole or in part, or host
                            Products for use by others.
                            Version: December 2022 2 (6)"""
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Legal Expert
 [INFO]: == Starting Task: AYESA just reached out with a super important ask about the agreement below:
Question: Extract all unfair terms from the GitHub terms of service.

Agreement: Version: December 2022 1 (6)
                            GitHub Customer Agreement
                            This Agreement consists of the General Terms, the applicable Product Specific Terms and any additional terms
                            GitHub presents when an Order is placed. The Agreement takes effect when Customer accepts the General
                            Terms, and the individual who accepts these General Terms represents that they are authorized to enter into
                            this Agreement on behalf of Customer.
                            GitHub General Terms
                            These General Terms apply to all of Customer’s Orders under this Agreement. Capitalized terms have the
                            meanings given

In [36]:
from IPython.display import Markdown
Markdown(result)

The LexGLUE tool has identified several clauses from the GitHub Customer Agreement, with particular attention drawn to clause 1.6 regarding Previews, which are provided "AS-IS", "WITH ALL FAULTS" and "AS AVAILABLE", allowing GitHub the right to change or discontinue Previews at any time without notice. This could be considered potentially unfair as it provides no assurance of service continuity or quality, leaving customers without recourse. The rest of the clauses did not register specific unfairness concerns from the LexGLUE tool.